# Topic Modeling - Latent Dirichlet Allocation & Non-Negative Matric Factorization

## Data

NPR (National Public Radio) articles [www.npr.org](http://www.npr.org)

In [12]:
import pandas as pd

In [13]:
npr = pd.read_csv('npr.csv')

In [14]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


We don't have the topic of the articles! Use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [16]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [17]:
dtm = cv.fit_transform(npr['Article'])

In [18]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## LDA

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [10]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## Showing Stored Words

In [11]:
len(cv.get_feature_names())

54777

In [12]:
import random

In [13]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

coldblooded
odham
cropped
atoms
mischievous
democratized
congee
dwayne
fearsome
adirondack


In [15]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

invades
pimple
responses
mcdonald
murat
cosmologist
carrier
ruby
evacuate
recruiting


### Showing Top Words Per Topic

In [16]:
len(LDA.components_)

7

In [17]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [18]:
len(LDA.components_[0])

54777

In [19]:
single_topic = LDA.components_[0]

In [20]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [21]:
# Word least representative of this topic
single_topic[18302]

0.14285714309286987

In [22]:
# Word most representative of this topic
single_topic[42993]

6247.2455105211

In [23]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [24]:
top_word_indices = single_topic.argsort()[-10:]

In [25]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


These look like business articles...

View top 15 words for the 7 topics found.

In [26]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think',

### Attaching Discovered Topic Labels to Original Articles

In [27]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [28]:
dtm.shape

(11992, 54777)

In [29]:
len(npr)

11992

In [30]:
topic_results = LDA.transform(dtm)

In [31]:
topic_results.shape

(11992, 7)

In [32]:
topic_results[0]

array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [33]:
topic_results[0].round(2)

array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [34]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [35]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [36]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 0])

In [37]:
npr['Topic'] = topic_results.argmax(axis=1)

In [39]:
npr.head(50)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",3
9,"Eighteen years ago, on New Year’s Eve, David F...",2


# NMF

In [19]:
from sklearn.decomposition import NMF

In [20]:
nmf_model = NMF(n_components=7,random_state=42)

In [21]:
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

## Preprocessing

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [25]:
dtm = tfidf.fit_transform(npr['Article'])

In [26]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## Display Topics

In [27]:
len(tfidf.get_feature_names())

54777

In [29]:
import random

In [30]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

raspberries
careens
percent
thrones
receiving
brett
indicates
captures
caitlyn
gabbana


In [31]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

campaigns
extradite
guesswork
abs
anjuli
romanian
dystopias
bullshit
skim
comprehensively


In [32]:
len(nmf_model.components_)

7

In [33]:
len(nmf_model.components_[0])

54777

In [37]:
single_topic = nmf_model.components_[0]

In [38]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([17319, 18349, 40552, ..., 26752, 36283, 28659])

In [39]:
# Word least representative of this topic
single_topic[18302]

0.1380475035853112

In [40]:
# Word most representative of this topic
single_topic[42993]

0.0

In [41]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([15008, 53211, 49459, 21073, 39848, 27439, 49183, 26752, 36283,
       28659])

In [42]:
top_word_indices = single_topic.argsort()[-10:]

In [43]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

don
way
time
going
really
know
think
just
people
like


In [44]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'lot', 'make', 'things', 've', 'don', 'way', 'time', 'going', 'really', 'know', 'think', 'just', 'people', 'like']


THE TOP 15 WORDS FOR TOPIC #1
['presidential', 'republicans', 'comey', 'administration', 'obama', 'new', 'election', 'republican', 'white', 'donald', 'house', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['government', 'say', 'city', 'day', 'university', 'year', 'water', 'food', 'time', 'just', 'new', 'years', 'like', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['according', 'reports', 'news', 'department', 'new', 'law', 'time', 'government', 'house', 'npr', 'court', 'told', 'president', 'police', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['country', 'election', 'new', 'states', 'vote', 'party', 'obama', 'democratic', 'voters', 'percent', 'campaign', 'hillary', 'state', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['new', 'percent', 'year', 'parents', 'private', 'kids', 'children', 'student',

In [45]:
topic_results = nmf_model.transform(dtm)

In [46]:
topic_results.shape

(11992, 7)

In [47]:
topic_results[0].round(2)

array([0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  ])

In [48]:
topic_results[0].argmax()

1

### Combining with Original Data

In [49]:
npr['Topic'] = topic_results.argmax(axis=1)

In [50]:
npr.head(50)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",5
5,I did not want to join yoga class. I hated tho...,0
6,With a who has publicly supported the debunk...,6
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",2
9,"Eighteen years ago, on New Year’s Eve, David F...",0
